# Python imports

In [1]:
# Show plots as part of the notebook (this is a Jupyter-specific operation)
%matplotlib inline
import time
import matplotlib.pyplot as plt

# Standard library packages
import os

# Import Pandas and Seaborn
import pandas as pd
import seaborn as sns

# Import Biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastxCommandline
from Bio.Blast.Applications import NcbiblastnCommandline

In [2]:
blastn -help

NameError: name 'blastn' is not defined

In [21]:
help(NcbiblastnCommandline)

Help on class NcbiblastnCommandline in module Bio.Blast.Applications:

class NcbiblastnCommandline(_NcbiblastMain2SeqCommandline)
 |  NcbiblastnCommandline(cmd='blastn', **kwargs)
 |  
 |  Wrapper for the NCBI BLAST+ program blastn (for nucleotides).
 |  
 |  With the release of BLAST+ (BLAST rewritten in C++ instead of C), the NCBI
 |  replaced the old blastall tool with separate tools for each of the searches.
 |  This wrapper therefore replaces BlastallCommandline with option -p blastn.
 |  
 |  For example, to run a search against the "nt" nucleotide database using the
 |  FASTA nucleotide file "m_code.fasta" as the query, with an expectation value
 |  cut off of 0.001, saving the output to a file in XML format:
 |  
 |  >>> from Bio.Blast.Applications import NcbiblastnCommandline
 |  >>> cline = NcbiblastnCommandline(query="m_cold.fasta", db="nt", strand="plus",
 |  ...                               evalue=0.001, out="m_cold.xml", outfmt=5)
 |  >>> cline
 |  NcbiblastnCommandline(

In [ ]:
blastn -task blastn -query example/unknown.fa -db db/bacteria/bacteria_nucl -out example/blastn_bacteria.out

# Running and analyzing a local BLASTN search

In [24]:
cmd_blastn = NcbiblastnCommandline(query='July142020-ALL-R1.fasta',
                      out='output_blastn.tab',
                      outfmt=6,
                      db='db_ref/ref_db_16S_GreenAlgae',
                      num_threads = 64)

In [25]:
# Get a working command-line
print(cmd_blastn)

blastn -out output_blastn.tab -outfmt 6 -query July142020-ALL-R1.fasta -db db_ref/ref_db_16S_GreenAlgae -num_threads 64


In [28]:
%%time
cmd_blastn()

CPU times: user 625 ms, sys: 436 ms, total: 1.06 s
Wall time: 3h 22min 16s


('', '')

In [ ]:
# Run BLASTX, and catch STDOUT/STDERR
# !! Do not execute cell if skipping computation !!
stdout, stderr = cmd_blastn()

# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

In [29]:
# !! If you are skipping computational steps, uncomment the line below !!
#blastout = os.path.join('prepped', 'kitasatospora', 'AMK19_00175_blastx_kitasatospora.tab')  # BLAST output

# Read BLASTX output
results = pd.read_csv("output_blastn.tab", sep="\t", header=None)

In [31]:
results.shape

(6289929, 12)

In [30]:
# Inspect results table
results.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,A00842:193:HMWFWDRXY:1:2101:18322:1094,NC_009630.1,83.146,89,15,0,32,120,1992,1904,1.170000e-15,82.4
1,A00842:193:HMWFWDRXY:1:2101:18322:1094,EF463011.1,83.146,89,15,0,32,120,1992,1904,1.170000e-15,82.4
2,A00842:193:HMWFWDRXY:1:2101:18322:1094,NC_038217.1,95.238,42,2,0,27,68,187950,187909,3.270000e-11,67.6
3,A00842:193:HMWFWDRXY:1:2101:18322:1094,MN201586.1,95.238,42,2,0,27,68,155398,155439,3.270000e-11,67.6
4,A00842:193:HMWFWDRXY:1:2101:18322:1094,KX761577.1,95.238,42,2,0,27,68,187950,187909,3.270000e-11,67.6


In [32]:
# Define column headers
headers = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore']

# Assign headers
results.columns = headers

# Inspect modified table
results.head()

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,A00842:193:HMWFWDRXY:1:2101:18322:1094,NC_009630.1,83.146,89,15,0,32,120,1992,1904,1.170000e-15,82.4
1,A00842:193:HMWFWDRXY:1:2101:18322:1094,EF463011.1,83.146,89,15,0,32,120,1992,1904,1.170000e-15,82.4
2,A00842:193:HMWFWDRXY:1:2101:18322:1094,NC_038217.1,95.238,42,2,0,27,68,187950,187909,3.270000e-11,67.6
3,A00842:193:HMWFWDRXY:1:2101:18322:1094,MN201586.1,95.238,42,2,0,27,68,155398,155439,3.270000e-11,67.6
4,A00842:193:HMWFWDRXY:1:2101:18322:1094,KX761577.1,95.238,42,2,0,27,68,187950,187909,3.270000e-11,67.6


In [33]:
# Show a summary of the results table data
results.describe()

,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
count,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06,6.289929e+06
mean,9.049531e+01,1.061238e+02,9.981368e+00,1.038350e+00,1.592050e+01,1.204425e+02,3.392174e+04,3.392104e+04,6.004135e-09,1.330045e+02
std,5.149892e+00,3.682165e+01,6.816648e+00,1.803333e+00,2.431531e+01,3.415107e+01,9.292986e+04,9.292980e+04,1.133176e-06,4.212917e+01
min,7.337700e+01,2.500000e+01,0.000000e+00,0.000000e+00,1.000000e+00,2.800000e+01,1.000000e+00,1.000000e+00,3.860000e-75,4.170000e+01
25%,8.686100e+01,7.500000e+01,4.000000e+00,0.000000e+00,1.000000e+00,9.700000e+01,4.840000e+02,4.840000e+02,1.460000e-39,1.000000e+02
50%,9.059800e+01,1.090000e+02,9.000000e+00,0.000000e+00,3.000000e+00,1.350000e+02,1.116000e+03,1.107000e+03,4.120000e-30,1.300000e+02
75%,9.440600e+01,1.420000e+02,1.500000e+01,2.000000e+00,2.200000e+01,1.510000e+02,4.466900e+04,4.466900e+04,3.230000e-21,1.610000e+02
max,1.000000e+02,1.770000e+02,3.900000e+01,2.000000e+01,1.240000e+02,1.510000e+02,4.263916e+06,4.263944e+06,2.000000e-03,2.790000e+02


In [34]:
# Show all subject matches
print(results['subject'])

0          NC_009630.1
1           EF463011.1
2          NC_038217.1
3           MN201586.1
4           KX761577.1
              ...     
6289924     KX602105.1
6289925     KX602101.1
6289926     KX602099.1
6289927     KX602074.1
6289928     KX602081.1
Name: subject, Length: 6289929, dtype: object


In [ ]:
# Create a new column describing how long the alignment is on the query sequence
qaln_length = abs(results['query_end'] - results['query_start']) + 1
print(qaln_length)

In [ ]:
# Add qaln_length to the results table as a new column
results['qaln_length'] = qaln_length
results.head()

In [ ]:
# Create a scatterplot
results.plot.scatter('pc_identity', 'e_value')
plt.title("E value vs %identity");              # add a title to the plot

In [ ]:
# SOLUTION - EXERCISE 01
# !! Do not execute this cell if skipping computational step !!

# We can reuse the directories and db, but need to define new input/output filenames
query = os.path.join(datadir, 'lantibiotic.fasta')                                   # query sequence(s)
blastout = os.path.join(outdir, 'lantibiotic_blastx_kitasatospora.tab')              # BLAST output

# Create command-line for BLASTX
cmd_blastx = NcbiblastxCommandline(query=query, out=blastout, outfmt=6, db=db)

# Run BLASTX, and catch STDOUT/STDERR
stdout, stderr = cmd_blastx()

# Check STDOUT, STDERR
print("STDOUT: %s" % stdout)
print("STDERR: %s" % stderr)

In [ ]:
# !! Uncomment the line below, if skipping computational step !!
# blastout = os.path.join('prepped', 'kitasatospora', 'lantibiotic_blastx_kitasatospora.tab')

# Read BLASTX output, and reuse the column headers defined earlier
results = pd.read_csv(blastout, sep="\t", header=None)
results.columns = headers

# Create a scatterplot
results.plot.scatter('bitscore', 'pc_identity')
plt.title("%identity vs bitscore");  